In [ ]:
%pip -q install transformer_lens
%pip -q install circuitsvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.3 MB/s eta 0:00:00
  

## Import Libraries

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

import einops
#from fancy_einsum import einsum
import os
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache


device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [ ]:
# Define the location to save the model, using a relative path
PTH_LOCATION = "workspace/_scratch/grokking_xor1.pth"

# Create the directory if it does not exist
os.makedirs(Path(PTH_LOCATION).parent, exist_ok=True)

## DATASET

In [ ]:
from itertools import product

def generate_xor_dataset(bit_length=1, n_samples=None, device='cpu', seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
    if n_samples is None:
        all_pairs = list(product([0, 1], repeat=2 * bit_length))
        #random.shuffle(all_pairs)
        inputs = torch.tensor(all_pairs, dtype=torch.long, device=device)
    else:
        inputs = torch.randint(
            0, 2, size=(n_samples, 2 * bit_length), dtype=torch.long, device=device
        )

    a = inputs[:, :bit_length]
    b = inputs[:, bit_length:]
    targets = torch.bitwise_xor(a, b)

    return inputs, targets
seed = 598
dataset, labels = generate_xor_dataset(bit_length=4, device=device, seed=seed)

In [ ]:
print(dataset.shape)
print(labels.shape)
print(dataset)
print(labels)

torch.Size([256, 8])
torch.Size([256, 4])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')
tensor([[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        ...,
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]], device='cuda:0')


## Config

In [ ]:
frac_train = 0.115
# Optimizer config
lr = 1e-3
wd = 1
betas = (0.9, 0.98)

num_epochs = 30000
checkpoint_every = 100
n_samples = 256
bit_length = 4

In [ ]:
torch.manual_seed(seed)
indices = torch.randperm(n_samples)
cutoff = int(n_samples*frac_train)
train_indices = indices[:cutoff]
test_indices = indices[cutoff:]

train_data = dataset[train_indices]
train_labels = labels[train_indices]
test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(train_data[:5])
print(train_labels[:5])
print(train_data.shape)
print(test_data[:5])
print(test_labels[:5])
print(test_data.shape)

tensor([[0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 1, 0],
        [1, 0, 0, 1, 1, 1, 0, 1],
        [1, 1, 1, 0, 0, 1, 0, 1],
        [0, 1, 1, 0, 1, 1, 1, 0]], device='cuda:0')
tensor([[1, 1, 1, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 1, 1],
        [1, 0, 0, 0]], device='cuda:0')
torch.Size([29, 8])
tensor([[0, 1, 0, 1, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 0, 0, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 1],
        [0, 1, 1, 1, 0, 0, 1, 1]], device='cuda:0')
tensor([[1, 1, 1, 0],
        [1, 0, 0, 0],
        [1, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0]], device='cuda:0')
torch.Size([227, 8])


## Model

In [ ]:
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 2,
    d_model = 64,
    d_head = 32,
    d_mlp = 256,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=2,
    d_vocab_out=2,
    n_ctx=8,
    init_weights=True,
    device=device,
    seed = 999,
)

In [ ]:
model = HookedTransformer(cfg)

In [ ]:
# Disable the biases, as we don't need them for this task and it makes things easier to interpret.

for name, param in model.named_parameters():
    if "b_" in name:
        param.requires_grad = False

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

In [ ]:
logits = model(train_data)
print(logits.shape)
log_probs = logits[:,-1,:].log_softmax(dim=-1)
print(log_probs.shape)
print(train_labels.shape)
print(train_labels.unsqueeze(-1).shape)

torch.Size([29, 8, 2])
torch.Size([29, 2])
torch.Size([29, 4])
torch.Size([29, 4, 1])


In [ ]:
def loss_fn(logits, labels):
    if len(logits.shape)==3:
      logits = logits[:,4:8,:]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels.unsqueeze(-1))
    return -correct_log_probs.mean()
train_logits = model(train_data)
train_loss = loss_fn(train_logits, train_labels)
print(train_loss)
test_logits = model(test_data)
test_loss = loss_fn(test_logits, test_labels)
print(test_loss)

tensor(0.7264, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)
tensor(0.7047, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)


## Training

In [ ]:
train_losses = []
test_losses = []
model_checkpoints = []
checkpoint_epochs = []
for epoch in tqdm.tqdm(range(num_epochs)):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    train_losses.append(train_loss.item())

    optimizer.step()
    optimizer.zero_grad()

    with torch.inference_mode():
        test_logits = model(test_data)
        test_loss = loss_fn(test_logits, test_labels)
        test_losses.append(test_loss.item())

    if ((epoch+1)%checkpoint_every)==0:
        checkpoint_epochs.append(epoch)
        model_checkpoints.append(copy.deepcopy(model.state_dict()))
        print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")

  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 99 Train Loss 0.030021441755825495 Test Loss 0.8303669517703793
Epoch 199 Train Loss 0.00037066741755052544 Test Loss 1.1042912855176592
Epoch 299 Train Loss 0.00012115187137547482 Test Loss 1.2339163216111297
Epoch 399 Train Loss 3.830071327049715e-05 Test Loss 1.3786713926666785
Epoch 499 Train Loss 1.2400633690129767e-05 Test Loss 1.525739116083395
Epoch 599 Train Loss 4.011186973046374e-06 Test Loss 1.6780897942100317
Epoch 699 Train Loss 1.327954423996522e-06 Test Loss 1.8298483338922962
Epoch 799 Train Loss 4.7346538585037905e-07 Test Loss 1.9673466589305477
Epoch 899 Train Loss 1.7483924014203614e-07 Test Loss 2.080839493487997
Epoch 999 Train Loss 7.025665648776926e-08 Test Loss 2.1653758422823217
Epoch 1099 Train Loss 3.1239919446831464e-08 Test Loss 2.2037760632513987
Epoch 1199 Train Loss 1.5496999282827136e-08 Test Loss 2.1918121112687063
Epoch 1299 Train Loss 8.45381657375591e-09 Test Loss 2.137310156716523
Epoch 1399 Train Loss 5.69193332523595e-09 Test Loss 2.04328

In [ ]:
%pip install git+https://github.com/neelnanda-io/neel-plotly.git
from neel_plotly.plot import line

  Cloning https://github.com/neelnanda-io/neel-plotly.git to /tmp/pip-req-build-m0si6srs
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly.git /tmp/pip-req-build-m0si6srs
  Resolved https://github.com/neelnanda-io/neel-plotly.git to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done
  Created wheel for neel_plotly: filename=neel_plotly-0.0.0-py3-none-any.whl size=10188 sha256=ed727d03e1ec451ba8b4a0e2199eea084869af21e9e83cd96358fd62bb13c797
  Stored in directory: /tmp/pip-ephem-wheel-cache-az0v_m_b/wheels/8b/67/f8/8a94ccda37eea5ac48d867ea4bfa87a4431501b03cfe31f7c2
Successfully built neel_plotly


In [ ]:
subtitle = f"(frac_train={frac_train}, lr={lr}, wd={wd})"

fig = line(
    [train_losses[::100], test_losses[::100]],
    x=np.arange(0, len(train_losses), 100),
    xaxis="Epoch",
    yaxis="Loss",
    log_y=True,
    # add the hyper-params right in the title
    title=f"Training Curve for XOR {subtitle}",
    line_labels=["train", "test"],
    toggle_x=True,
    toggle_y=True,
)

In [ ]:
# save the trained model

torch.save(
    {
        "model":model.state_dict(),
        "config": model.cfg,
        "checkpoints": model_checkpoints,
        "checkpoint_epochs": checkpoint_epochs,
        "test_losses": test_losses,
        "train_losses": train_losses,
        "train_indices": train_indices,
        "test_indices": test_indices,
    },
    PTH_LOCATION)

In [ ]:
# 1) Delete the old checkpoint file, if you saved one:
if os.path.exists(PTH_LOCATION):
    os.remove(PTH_LOCATION)
    print(f"Removed checkpoint file at {PTH_LOCATION}")

# 2) Clear out your history lists:
train_losses = []
test_losses = []
model_checkpoints = []
checkpoint_epochs = []
print("Cleared loss and checkpoint history.")

Cleared loss and checkpoint history.
